In [79]:
import pandas as pd
from datetime import date, timedelta
from tqdm import tqdm
import unidecode
import json
from itertools import groupby 
from collections import Counter

In [5]:
año, mes, dia = "2020-04-20".split("-")

In [6]:
dia

'20'

In [7]:
sdate = date(int(año), int(mes), int(dia))   # start date
edate = date.today()   # end date
delta = edate - sdate       # as timedelta

dates = []
for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    dates.append(str(day))

In [8]:
dates

['2020-04-20',
 '2020-04-21',
 '2020-04-22',
 '2020-04-23',
 '2020-04-24',
 '2020-04-25',
 '2020-04-26',
 '2020-04-27',
 '2020-04-28',
 '2020-04-29',
 '2020-04-30',
 '2020-05-01',
 '2020-05-02',
 '2020-05-03',
 '2020-05-04',
 '2020-05-05',
 '2020-05-06',
 '2020-05-07',
 '2020-05-08',
 '2020-05-09',
 '2020-05-10',
 '2020-05-11',
 '2020-05-12',
 '2020-05-13',
 '2020-05-14',
 '2020-05-15',
 '2020-05-16',
 '2020-05-17',
 '2020-05-18',
 '2020-05-19',
 '2020-05-20',
 '2020-05-21',
 '2020-05-22',
 '2020-05-23',
 '2020-05-24',
 '2020-05-25',
 '2020-05-26',
 '2020-05-27',
 '2020-05-28',
 '2020-05-29',
 '2020-05-30',
 '2020-05-31',
 '2020-06-01',
 '2020-06-02',
 '2020-06-03',
 '2020-06-04',
 '2020-06-05',
 '2020-06-06',
 '2020-06-07',
 '2020-06-08',
 '2020-06-09',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12']

In [72]:
dicc_names = {
    "No. Caso": "Caso",
    "ID de caso": "Caso",
    "Tipo*": "Tipo",
    "País de procedencia": "Pais de procedencia",
    "Pais de procedencia\t": "Pais de procedencia",
    "Pais a donde viajo  1": "Pais de procedencia",
    "Ciudad de ubicación": "Ciudad",
    "Ciudad/Municipio": "Ciudad",
    "Atención**": "Ubicacion",
    "Ubicación": "Ubicacion",
    "Departamento o Distrito": "Departamento",
    "fuente o tipo de contagio": "Tipo",
    "Código ciudad": "divipola",
    "Fecha de diagnóstico": "Fecha de diagnostico",
    "Fechadiagnostico": "Fecha de diagnostico"
}

In [132]:
all_data = pd.DataFrame()   #descomentar si bid_df no existe

for date in tqdm(dates):
    try:
        link =f'https://www.ins.gov.co/BoletinesCasosCOVID19Colombia/{date}.xlsx'
        data = pd.read_excel(link)
    except:
        print("ended in " + date)
        break
    data["fecha_boletin"] = date
    data = data.rename(columns=dicc_names)
    data["Fecha de diagnostico"] = pd.to_datetime(data["Fecha de diagnostico"], errors="coerce")
#     data = data[data["Fecha de diagnostico"] > sdate] 
    
    
    data["Estado"] = data["Estado"].str.lower().str.strip()
    data["Ubicacion"] = data["Ubicacion"].str.lower().str.strip()
    
    data = data.loc[:,["Caso", "Estado", "Ubicacion"]]
    all_data = all_data.append(data)
    
#     estado = data["Estado"].value_counts().to_frame(date).T.drop("recuperado", axis=1, errors='ignore')
#     ubicacion = data["Ubicacion"].value_counts().to_frame(date).T.drop("fallecido", axis=1) # fallecido ya esta en estado
    
#     est_ubi = estado.merge(ubicacion, left_index=True, right_index=True)
#     medellin_estado = medellin_estado.append(est_ubi)




 94%|█████████▍| 51/54 [04:24<00:15,  5.18s/it]

ended in 2020-06-10


In [124]:
ids_muertes = all_data[all_data["Ubicacion"] == "fallecido"].Caso.values

In [125]:
muertos = all_data[all_data["Caso"].isin(ids_muertes)]

In [126]:
muertos

,Caso,Estado,Ubicacion
3990,3991,moderado,hospital
4000,4001,grave,hospital uci
4001,4002,grave,hospital uci
4002,4003,leve,casa
4005,4006,moderado,hospital
...,...,...,...
40182,40223,fallecido,fallecido
40560,40601,fallecido,fallecido
40696,40737,fallecido,fallecido
41635,41676,fallecido,fallecido


In [128]:
rutas = []
for id_m in set(ids_muertes):
    persona = muertos[muertos["Caso"] == id_m]
    ubicaciones = persona["Ubicacion"].values
    res = [str(i[0]) for i in groupby(ubicaciones)]
    rutas.append("-".join(res))


In [129]:
df_rutas = pd.DataFrame(Counter(rutas), index=[0]).T.rename(columns={0:"cuenta"}).sort_values("cuenta", ascending=False)

In [130]:
len(set(ids_muertes))

1426

In [131]:
df_rutas["cuenta"].head(10)

hospital-fallecido                      270
casa-fallecido                          185
casa-fallecido-casa-recuperado          178
fallecido                               106
hospital uci-fallecido                   85
fallecido-casa-fallecido                 66
hospital-fallecido-casa-fallecido        61
casa-fallecido-casa-fallecido            51
hospital-hospital uci-fallecido          32
casa-recuperado-fallecido-recuperado     27
Name: cuenta, dtype: int64